In [ ]:
import numpy as np
import xfab.tools
import os
import numpy as np
import matplotlib.pyplot as plt
from xfab.symmetry import Umis
import xfab.symmetry
from collections import deque
from scipy.ndimage import binary_fill_holes
from scipy.spatial.transform import Rotation
import matplotlib
import torch
import skimage
import scipy.ndimage

In [ ]:
keys = ['layer_top', 'layer_center', 'layer_bottom']
dirs = [os.path.join('..', dir, 'test_run') for dir in keys]
sample = {}
sample['layer_top']    = np.fliplr(  np.load(os.path.join(dirs[0], 'sample_mask_layer_top.npy'))[5:-5, 5:-5]  )
sample['layer_center'] = np.fliplr(  np.load(os.path.join(dirs[1], 'sample_mask_layer_center.npy'))[2:-2, 2:-2] )
sample['layer_bottom'] = np.fliplr(  np.load(os.path.join(dirs[2], 'sample_mask_layer_bottom.npy'))[2:-2, 2:-2] )

gmap = {}
for key in keys:
    gmap[key] = np.load('../'+key+'/test_run/gmap.npy', allow_pickle=True) # gmap[i,j] = ubis, npeaks

stress = {}
stress['layer_top'] = np.load('../layer_top/test_run/stress_map_top_layer_refined_com_corrected.npy')
stress['layer_center'] = np.load('../layer_center/test_run/stress_map_central_layer_refined_com_corrected.npy')
stress['layer_bottom'] = np.load('../layer_bottom/test_run/stress_map_bottom_layer_refined_com_corrected.npy')

strain = {}
strain['layer_top'] = np.load('../layer_top/test_run/strain_map_top_layer_refined_com_corrected.npy')
strain['layer_center'] = np.load('../layer_center/test_run/strain_map_central_layer_refined_com_corrected.npy')
strain['layer_bottom'] = np.load('../layer_bottom/test_run/strain_map_bottom_layer_refined_com_corrected.npy')

ubi_map, X, Y, ipf_map, xypf_map = {},{},{},{}, {}
for i,key in enumerate(keys):
    X[key] = np.load( os.path.join( dirs[i], 'X_coord_gmap.npy' ) )
    Y[key] = np.load( os.path.join( dirs[i], 'Y_coord_gmap.npy' ) )
    ubi_map[key]    = np.load(os.path.join(dirs[i], 'ubi_median_filt_s6.npy'))
    ipf_map[key]    = np.load(os.path.join(dirs[i], 'ipf_median_filt_s6.npy'))
    xypf_map[key]    = np.load(os.path.join(dirs[i], 'xypf_median_filt_s6.npy'))

n = np.max( stress['layer_top'].shape + stress['layer_center'].shape + stress['layer_bottom'].shape )
stress_3D = np.zeros( (3, n,n,3,3) )
for i,key in enumerate(keys):
    k = stress[key].shape[0]
    pad = (n-k)//2
    if pad==0: stress_3D[i] = stress[key]
    else:stress_3D[ i, pad:-pad, pad:-pad] = stress[key]

kam = {}
for i,key in enumerate(keys):
    kam[key] = np.load(key + '_kam.npy')

grain_boundaries = {}
for i,key in enumerate(keys):
    grain_boundaries[key] = np.load(key + '_grain_boundaries.npy')

hydrostatic, devatoric, effective, J2, J3, triaxial, L, theta = {}, {}, {}, {}, {}, {}, {}, {}
for key in keys:
    hydrostatic[key] = ( stress[key][:,:,0,0] + stress[key][:,:,1,1] + stress[key][:,:,2,2] ) / 3.
    devatoric[key] = stress[key].copy()
    devatoric[key][:,:,0,0]   -=  hydrostatic[key]
    devatoric[key][:,:,1,1]   -=  hydrostatic[key]
    devatoric[key][:,:,2,2]   -=  hydrostatic[key]
    J2[key]    = ( np.sum(devatoric[key]*devatoric[key], axis=(-2,-1)) )/2.
    effective[key] = np.sqrt(3*J2[key]) 

    J3[key] = np.zeros_like(J2[key])
    for i in range(stress[key].shape[0]):
        for j in range(stress[key].shape[1]):
            J3[key][i,j] = np.linalg.det(devatoric[key][i,j])

    denom = (effective[key] + 1e-32).copy()
    triaxial[key] = ( hydrostatic[key] / denom ) * (effective[key]!=0)
    L[key] = (effective[key]!=0) * (-27/2.)*J3[key]/( denom**3 )
    theta[key] = np.degrees( np.arccos(-L[key]) / 3. )

In [ ]:
channels_stats = np.array( [len(g[0]) for g in gmap[key][sample[key]].flatten() for key in keys] )
exponent = len(channels_stats) / np.emath.logn(channels_stats.mean(), 10)
print('mean:', channels_stats.mean(), '  std:', channels_stats.std(), 'exponent: ', exponent)
plt.figure(figsize=(8,5))
plt.hist(channels_stats, bins=np.linspace(-0.5, 11, 13))
plt.show()

In [ ]:
xx  = X[key][sample[key]]
yy  = Y[key][sample[key]]
xb = X[key][grain_boundaries[key]]
yb = Y[key][grain_boundaries[key]]
dx = np.array( [np.min( np.sqrt( (x-xb)**2 + (y-yb)**2 ) / 3.0 ) for x,y in zip(xx, yy)] )
ss = effective[key][sample[key]] / 1e6

angle = []
zhat= np.array([0,0,1])
for ubi in ubi_map[key][sample[key]] :
    cell = np.linalg.inv(ubi).T
    qq = cell @ zhat / np.linalg.norm(cell, axis=0)
    tt = np.arccos(qq)
    angle.append( np.degrees( tt ).min() )
angle = np.array(angle)


In [ ]:
def flood_fill( orientation_map, 
                seed_point, 
                footprint, 
                crystal_system, 
                local_disorientation_tolerance,
                global_disorientation_tolerance, 
                mask=None, 
                background_value=np.nan,
                fill_holes=False,
                max_grains = 99,
                min_grain_size = 0,
                verbose=False
                ):
    assert footprint.shape[0]%2!=0
    assert footprint.shape[1]%2!=0

    M, N = orientation_map.shape[0], orientation_map.shape[1]
    segmentation = np.zeros((M,N))
    skipps = np.ones((M,N), dtype=bool)
    label = 1
    segmentation[seed_point[0], seed_point[1]:seed_point[1] + 3] = label
    done = False
    iteration = 0
    while( not done and iteration < max_grains ):
        rows, cols = np.where( mask*(segmentation==0)*skipps )
        if len(rows)>0:
            n = np.random.randint( 0, len(rows) )
            seed_point = ( rows[n], cols[n] )
            grain_mask = _flood(orientation_map, 
                                seed_point, 
                                footprint, 
                                crystal_system, 
                                local_disorientation_tolerance,
                                global_disorientation_tolerance,
                                mask)
            if fill_holes: grain_mask = binary_fill_holes(grain_mask)
            if np.sum(grain_mask) > min_grain_size:
                segmentation[grain_mask] = label
                label += 1
            else:
                skipps[grain_mask] = False
            iteration += 1
            if verbose:
                print('Iteration ', iteration, ', found grain with : ', np.sum(grain_mask), ' voxels @, seed_point, ', np.sum(grain_mask))
        else:
            done = True

    segmentation[segmentation==0] = background_value
    return segmentation

def _align( us, crystal_system, ref=None ):
    """Align a set of orientation matrices as closely as possible given a lattice symmetry.
    """
    rot = xfab.symmetry.rotations(crystal_system)
    if ref is None: ref = us[0]
    newu = np.zeros_like(us)
    mmis = []
    for i,u in enumerate(us):
        mis = xfab.symmetry.Umis( u, ref, crystal_system )
        rotindex = np.argmin(mis[:,1])
        newu[i,:,:] =  u @ rot[rotindex]
        mmis.append( mis[:,1].min() )
    return newu, np.array(mmis)


def _flood( orientation_map, 
            seed_point, 
            footprint, 
            crystal_system,
            local_disorientation_tolerance,
            global_disorientation_tolerance,
            mask ):
    m = footprint.shape[0] // 2
    n = footprint.shape[1] // 2
    flood_mask = np.zeros((orientation_map.shape[0], orientation_map.shape[1]), dtype=bool)

    i, j = seed_point
    flood_mask[i, j] = True
    if mask is not None and mask[i, j]==0: raise ValueError('Seed point not in mask')
    
    unchartered_indices = deque([ seed_point ])
    while(  len(unchartered_indices) > 0 ):
        i, j = unchartered_indices.pop()
        U = orientation_map[i, j]
        
        if np.sum(flood_mask) < 20:
            us = _align( orientation_map[flood_mask,:,:], crystal_system )
            global_U = Rotation.from_matrix( us ).mean().as_matrix()

        for k in range(footprint.shape[0]):
            for l in range(footprint.shape[1]):
                if footprint[k, l]>0:
                    row = i - m + k
                    col = j - n + l
                    if not flood_mask[row, col] and mask is not None and mask[row, col]:
                        u = orientation_map[row, col]
                        misorientation = Umis( U, u, crystal_system )[:,1].min()
                        
                        if misorientation < local_disorientation_tolerance:
                            glob_misorientation = Umis( global_U, u, crystal_system )[:,1].min()
                            if glob_misorientation < global_disorientation_tolerance:
                                flood_mask[row, col] = True
                                unchartered_indices.appendleft(  ( row, col) )
    return flood_mask
def get_u_map(ubi_map, sample_mask):
    u_map = np.zeros_like(ubi_map)
    for i in range(ubi_map.shape[0]):
        for j in range(ubi_map.shape[1]):
            if sample_mask[i,j]:
                ubi = ubi_map[i,j]
                u = xfab.tools.ubi_to_u(ubi)
                u_map[i,j] = u
    return u_map



In [ ]:
footprint= np.array([[ 0, 0 ,1, 0, 0 ],
                        [ 0, 1, 1, 1, 0 ],
                        [ 1, 1, 1, 1, 1 ],
                        [ 0, 1, 1, 1, 0 ],
                        [ 0, 0, 1, 0, 0 ]])
orientation_maps = {}
for key in keys:
    orientation_maps[key] = get_u_map(ubi_map[key], sample[key])

In [ ]:
if 0:
    seg = {}
    for key in keys:
        print('')
        print(key)
        seg[key] = flood_fill( orientation_maps[key], 
                        seed_point=(200,200), 
                        footprint=footprint, 
                        crystal_system=7, 
                        local_disorientation_tolerance=4.0,
                        global_disorientation_tolerance=np.inf, 
                        mask=sample[key], 
                        background_value=np.nan,
                        fill_holes=False,
                        max_grains = 199,
                        min_grain_size = 0,
                        verbose=True
                        )
    np.save('segmented_grains.npy', seg)
else:
    seg = np.load('segmented_grains.npy', allow_pickle=True)[()]

In [ ]:
cc=0
for key in keys:
    ngrains = np.nanmax(seg[key]).astype(int)
    for i in range(1, ngrains+1):
        m = seg[key]==i 
        if m.sum()>25:
            cc+=1
print('Number fo z-grain-slices are : ', cc)

In [ ]:
def crystal_direction_cubic( ubi, axis ):
    hkl = np.dot( ubi, axis )
    # cubic symmetry implies:
    #      24 permutations of h,k,l
    #      one has abs(h) <= abs(k) <= abs(l)
    hkl = abs(hkl)
    hkl.sort()
    return hkl

def hkl_to_color_cubic( hkl ):
    """
    https://mathematica.stackexchange.com/questions/47492/how-to-create-an-inverse-pole-figure-color-map
        [x,y,z]=u⋅[0,0,1]+v⋅[0,1,1]+w⋅[1,1,1].
            These are:
                u=z−y, v=y−x, w=x
                This triple is used to assign each direction inside the standard triangle

    makeColor[{x_, y_, z_}] :=
         RGBColor @@ ({z - y, y - x, x}/Max@{z - y, y - x, x})
    """
    x,y,z = hkl
    assert x<=y<=z
    assert z>=0
    u,v,w = z-y, y-x, x
    m = max( u, v, w )
    r,g,b = u/m, v/m, w/m
    return (r,g,b)

def ubi_from_rgb_cubic( rgb, axes ):
    hkl = np.zeros_like(rgb)
    hkl[0,:] = rgb[2,:]
    hkl[1,:] = rgb[1,:] + hkl[0,:]
    hkl[2,:] = rgb[0,:] + hkl[1,:]
    # hkl = rgb @ axes
    ubi = ( hkl @ np.linalg.inv(axes) )
    if np.dot( np.cross(ubi.T[:,0], ubi.T[:,1]), ubi.T[:,2] ) < 0:
        ubi[:,2] *= -1
    return ubi

def hkl_to_pf_cubic( hkl ):
    x,y,z = hkl
    assert x<=y<=z
    assert z>=0
    m = np.sqrt((hkl**2).sum())
    return x/(z+m), y/(z+m)

def triangle(  ):
    """ compute a series of point on the edge of the triangle """
    xy = [ np.array(v) for v in ( (0,1,1), (0,0,1), (1,1,1) ) ]
    xy += [ xy[2]*(1-t) + xy[0]*t for t in np.linspace(0.1,1,15)]
    return np.array( [hkl_to_pf_cubic( np.array(p) ) for p in xy] )

In [ ]:
us = orientation_maps['layer_bottom'][seg['layer_bottom']==11].copy()
def _align( us, crystal_system, ref=None ):
    """Align a set of orientation matrices as closely as possible given a lattice symmetry.
    """
    rot = xfab.symmetry.rotations(crystal_system)
    if ref is None: ref = us[0]
    newu = np.zeros_like(us)
    mmis = []
    for i,u in enumerate(us):
        mis = xfab.symmetry.Umis( u, ref, crystal_system )
        rotindex = np.argmin(mis[:,1])
        newu[i,:,:] =  u @ rot[rotindex]
        mmis.append( mis[:,1].min() )
    return newu, np.array(mmis)

us, mis = _align(us, 7)
mean_u = Rotation.from_matrix(us).mean().as_matrix()
diffrot = mean_u.T @ us
diffrot = np.degrees(Rotation.from_matrix(diffrot).magnitude())

_, mis2 = _align( us, crystal_system=7, ref=mean_u )

mis.max(), diffrot.max(), mis2.max()

In [ ]:
fig1, ax1 = plt.subplots(1,1, figsize=(7,7))
c = triangle().T
ax1.axis('off')
ax1.axis('equal')
ax1.plot(c[0], c[1], 'k-', linewidth=3)

fig2, ax2 = plt.subplots(1,1, figsize=(7,7))
c = triangle().T
ax2.axis('off')
ax2.axis('equal')
ax2.plot(c[0], c[1], 'k-', linewidth=3)

fig3, ax3 = plt.subplots(1,1, figsize=(7,7))
c = triangle().T
ax3.axis('off')
ax3.axis('equal')
ax3.plot(c[0], c[1], 'k-', linewidth=3)

dzero_cell = [4.04, 4.04, 4.04, 90,90,90]
B0 = xfab.tools.form_b_mat(dzero_cell) / (np.pi*2)

for key in keys:
    print('\n', key)

    ngrains = np.nanmax(seg[key]).astype(int)

    macro_xypf = []
    macro_ipf = []

    for i in range(1, ngrains+1):
        m = seg[key]==i 
        if m.sum()>25:
            us = orientation_maps[key][m].copy()

            us, mis = _align(us, 7)
            mean_u = Rotation.from_matrix(us).mean().as_matrix()
            ubi_mean = np.linalg.inv( B0 @ mean_u )

            # make sure we accounted for symmetry correctly
            diffrot = mean_u.T @ us
            diffrot = np.degrees(Rotation.from_matrix(diffrot).magnitude())
            if diffrot.max() >= 45:
                print( diffrot.max(), m.sum(), i )
                raise ValueError()
            #

            _ipf, _xypf = [], []
            for j in range(3):
                axis = np.zeros((3,))
                axis[j]=1
                hkl = crystal_direction_cubic( ubi_mean, axis )
                _ipf.append( hkl_to_color_cubic( hkl ) )
                _xypf.append( hkl_to_pf_cubic( hkl ) )
            macro_ipf.append( _ipf )
            macro_xypf.append( _xypf )
    macro_xypf, macro_ipf = np.array(macro_xypf).swapaxes(-2,-1), np.array(macro_ipf).swapaxes(-2,-1)
    ax1.scatter( macro_xypf[:,0,2], macro_xypf[:,1,2], c='k', marker='o', alpha=.5, s=100, linewidth=0)
    ax2.scatter( macro_xypf[:,0,1], macro_xypf[:,1,1], c='k', marker='o', alpha=.5, s=100, linewidth=0)
    ax3.scatter( macro_xypf[:,0,0], macro_xypf[:,1,0], c='k', marker='o', alpha=.5, s=100, linewidth=0)

fig1.savefig('gallery/macro_ipf_scatter_Z.png', dpi=400, pad_inches=0.000001, bbox_inches='tight')
fig2.savefig('gallery/macro_ipf_scatter_Y.png', dpi=400, pad_inches=0.000001, bbox_inches='tight')
fig1.savefig('gallery/macro_ipf_scatter_X.png', dpi=400, pad_inches=0.000001, bbox_inches='tight')

plt.show()

In [ ]:
for key in keys:
    coords = np.argwhere(sample[key])
    x_min, y_min = coords.min(axis=0)
    x_max, y_max = coords.max(axis=0)
    #cropped = seg[key][x_min:x_max+1, y_min:y_max+1].copy()
    
    Xc = X[key][x_min-6:x_max+6, y_min-6:y_max+6]
    Yc = Y[key][x_min-6:x_max+6, y_min-6:y_max+6]
    
    cropped = sample[key][x_min-6:x_max+6, y_min-6:y_max+6].copy().astype(int)
    #cropped[:,:] = np.nan

    cc=1
    for i in range(499):
        m = seg[key][x_min-6:x_max+6, y_min-6:y_max+6]==i 
        if m.sum() > 50:#4200:
            cropped[m] = cc+1
            plt.annotate( str(i) , (Xc[m].mean()-5, Yc[m].mean()-5), c='k', size=12 )
            cc+=1
            print(i)
        else:
            pass
            #cropped[m]=np.nan
    plt.pcolormesh(Xc, Yc, cropped, cmap='viridis', vmin=0, vmax=cc)
    plt.axis('equal')
    plt.show()



In [ ]:
dx, ss, zz, mm = {},{},{},{}
for key in keys:
    xx  = X[key][sample[key]]
    yy  = Y[key][sample[key]]
    xb = X[key][grain_boundaries[key]]
    yb = Y[key][grain_boundaries[key]]
    dx[key] = np.array( [np.min( np.sqrt( (x-xb)**2 + (y-yb)**2 ) / 3.0 ) for x,y in zip(xx, yy)] )
    ss[key] = effective[key][sample[key]] / 1e6
    zz[key] = stress[key][sample[key], 2, 2] / 1e6
    mm[key] = hydrostatic[key][sample[key]] / 1e6 

In [ ]:
umap = {}
for key in keys:
    umap[key] = get_u_map(ubi_map[key], sample[key])

In [ ]:
colors = np.array( [
    [55,  126, 184],  #377eb8 
    [255, 127, 0],    #ff7f00
    [77,  175, 74],   #4daf4a
    [247, 129, 191],  #f781bf
    [166, 86,  40],   #a65628
    [152, 78,  163],  #984ea3
    [153, 153, 153],  #999999
    [228, 26,  28],   #e41a1c
    [222, 222, 0]     #dede00
] )
colors = colors / 255


grain_id = np.array(  [   [1, 7, 5],
                          [6, 5, 16],
                          [7, 2, 1],
                          [14, 19, 2],
                          [2, 11, 10]])

grain_id = np.array(  [   [1, 7, 5],
                          [6, 5, 16],
                          [7, 2, 1],
                          [14, 19, 2],
                          [2, 11, 10]])

key = keys[1]
plt.figure(figsize=(14,7))
cc=1
m = [None, 'o', '^', '*', 's', 'd']
for ii in range(5):
    Us = []

    vol = 0
    for jj, key in enumerate(keys):
        mask = ( seg[key] == grain_id[ii, jj] )
        print(X[key][mask].mean(), Y[key][mask].mean())
        Us.append( _align( umap[key][mask,:,:], crystal_system=7 ) )
        vol += mask.sum()
    Us = np.concatenate(Us, axis=0)
    ref = Rotation.from_matrix(Us).mean().as_matrix()
    mis = [ xfab.symmetry.Umis( u, ref, crystal_system=7 )[:,1].min() for u in Us ]
    h, b = np.histogram( mis, bins = np.arange(-0.25, 15.5, 0.5)   )
    print(kam[key][mask][ (kam[key][mask]<5.0)*(kam[key][mask]>0) ].mean())
    bc = (b[0:-1] + b[1:])/2.
    plt.plot(bc, h / vol, '--', c=colors[cc], marker=m[cc], label=cc )
    cc+=1
plt.legend()
plt.grid()
plt.show()